In [ ]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from feature_engine.encoding import CountFrequencyEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import mlflow
import dagshub
import optuna

In [ ]:
# reading csv files
train = pd.read_csv('../data/processed/train.csv').drop(columns='month')
test = pd.read_csv('../data/processed/test.csv').drop(columns='month')

In [ ]:
# taking input and output data apart

xtrain = train.drop(columns='total_pickups')
xtest = test.drop(columns='total_pickups')

ytrain = train['total_pickups'].copy()
ytest = test['total_pickups'].copy()

In [ ]:
xtrain.set_index('tpep_pickup_datetime', inplace=True)
xtest.set_index('tpep_pickup_datetime', inplace=True)

In [31]:
xtrain

,region,avg_pickups,day_of_week,lag_1,lag_2,lag_3,lag_4
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,0,457.0,4,470.0,508.0,496.0,186.0
2016-01-01 01:15:00,0,471.0,4,489.0,470.0,508.0,496.0
2016-01-01 01:30:00,0,469.0,4,466.0,489.0,470.0,508.0
2016-01-01 01:45:00,0,450.0,4,423.0,466.0,489.0,470.0
2016-01-01 02:00:00,0,418.0,4,372.0,423.0,466.0,489.0
...,...,...,...,...,...,...,...
2016-02-29 22:45:00,29,78.0,0,89.0,82.0,57.0,65.0
2016-02-29 23:00:00,29,81.0,0,86.0,89.0,82.0,57.0
2016-02-29 23:15:00,29,77.0,0,71.0,86.0,89.0,82.0


In [ ]:
# taking  a standard scaler to scale the continuous features

transformer = ColumnTransformer(
    [
        ('scaler', StandardScaler(), ['lag_1', 'lag_2', 'lag_3', 'lag_4', 'avg_pickups'])
    ],
    remainder='passthrough'
)
transformer.set_output(transform='pandas')

xtrain_scaled = transformer.fit_transform(xtrain)
xtest_scaled = transformer.transform(xtest)

col_names = [x.split('__', maxsplit=1)[1] for x in xtrain_scaled.columns]

# renaming the columns

xtrain_scaled.columns = col_names
xtest_scaled.columns = col_names

In [ ]:
# converting categorical feature to dtype "category"

xtrain_scaled['region']=xtrain_scaled['region'].astype('category')
xtrain_scaled['day_of_week'] = xtrain_scaled['day_of_week'].astype('category')

In [ ]:
def objective_MAPE(trial):
    """
    This function do Optuna HP tunining and calculate loss on basis of MAPE
    """
    encoder_type = trial.suggest_categorical("encoder_type", ['Freq', 'Count', 'OHE'])

    if encoder_type == 'OHE':
        encoder = ColumnTransformer(
            transformers=[
                (
                    'OHE', OneHotEncoder(drop='first', sparse_output=False),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    elif encoder_type == 'Count' : 
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Count Encoder', CountFrequencyEncoder(encoding_method='count'),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    else:
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Freq Encoder', CountFrequencyEncoder(encoding_method='frequency'),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    model_type = trial.suggest_categorical("model", ['RF', 'LR', 'DT', 'XGB'])

    if model_type == 'LR':
        model = LinearRegression()
    
    elif model_type == 'RF':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n_estimators", 50,150),
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
        )

    elif model_type == 'DT':
        model = DecisionTreeRegressor(
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
            )
    
    else:
        model = XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 500),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.01, 0.2),
            max_depth=trial.suggest_categorical("xgb_max_depth", [None, 3, 10]),
            random_state=42
        )
    
    pipe = Pipeline(
        [('encoder', encoder),
        ('model', model)]
    )

    pipe.fit(xtrain_scaled, ytrain)

    ypred = pipe.predict(xtest_scaled)

    return mean_absolute_percentage_error(ytest, ypred)

In [ ]:
study = optuna.create_study(direction='minimize')

study.optimize(objective_MAPE, n_trials=50)

[I 2026-01-15 17:50:41,495] A new study created in memory with name: no-name-92260f2d-5abc-452e-a399-8a29247febfa


[I 2026-01-15 17:50:42,136] Trial 0 finished with value: 0.35340837533274566 and parameters: {'encoder_type': 'Count', 'model': 'DT', 'rf_max_depth': 5}. Best is trial 0 with value: 0.35340837533274566.
[I 2026-01-15 17:50:48,942] Trial 1 finished with value: 0.9046038389205933 and parameters: {'encoder_type': 'OHE', 'model': 'XGB', 'xgb_n_estimators': 173, 'xgb_learning_rate': 0.01381116058216431, 'xgb_max_depth': 10}. Best is trial 0 with value: 0.35340837533274566.
[I 2026-01-15 17:50:49,890] Trial 2 finished with value: 0.35340837533274566 and parameters: {'encoder_type': 'OHE', 'model': 'DT', 'rf_max_depth': 5}. Best is trial 0 with value: 0.35340837533274566.
[I 2026-01-15 17:50:50,008] Trial 3 finished with value: 0.41562576325682016 and parameters: {'encoder_type': 'Count', 'model': 'LR'}. Best is trial 0 with value: 0.35340837533274566.
[I 2026-01-15 17:50:50,873] Trial 4 finished with value: 0.35340837533274566 and parameters: {'encoder_type': 'OHE', 'model': 'DT', 'rf_max_de

In [52]:
mlflow.set_tracking_uri('https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow')
dagshub.init(repo_owner='akshatsharma2407', repo_name='Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=5408b90c-aecc-406f-9ae6-eedbcf41ab92&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=da1e8e2001934835863defb8b14e270d0ff92571d599ac24ac285e8709dec2ea




Accessing as akshatsharma2407

Initialized MLflow to track repo "akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering"

Repository akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering initialized!

In [ ]:
# experimentation tracking

mlflow.set_experiment(experiment_name='Optuna Experiment')
for trial in study.trials:
    with mlflow.start_run(run_name=f'trial_{trial.number}'):
        mlflow.log_params(trial.params)

        if trial.value:
            mlflow.log_metric('objective', trial.value)
        
        mlflow.set_tag('state', trial.state.name)
        mlflow.set_tag('trial_number', trial.number)

2026/01/15 18:58:40 INFO mlflow.tracking.fluent: Experiment with name 'Optuna Experiment' does not exist. Creating a new experiment.


🏃 View run trial_0 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0/runs/f667bed54c72448b957d760f51fb29ca
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0
🏃 View run trial_1 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0/runs/82d1a8fd609d4eca882dccfe348ee807
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0
🏃 View run trial_2 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0/runs/5224a9940e1a4db1884c3c327b531ee1
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/0
🏃 View run trial_3 at: https://dagshub.com/akshatsharma

In [ ]:
def SMAPE(y_true, y_pred):
    """
    custom SMAPE loss (Symmetrical Mean Absolute Percentage Error)
    """
    denominator = (np.abs(y_true) + np.abs(y_pred))/2
    diff = np.abs(y_true - y_pred) / denominator
    return np.mean(diff)

In [ ]:
def objective_SMAPE(trial):
    """
    Optimizing the algo with SMAPE as loss function
    """
    encoder_type = trial.suggest_categorical("encoder_type", ['Freq', 'Count', 'OHE'])

    if encoder_type == 'OHE':
        encoder = ColumnTransformer(
            transformers=[
                (
                    'OHE', OneHotEncoder(drop='first', sparse_output=False),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    elif encoder_type == 'Count' : 
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Count Encoder', CountFrequencyEncoder(encoding_method='count'),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    else:
        encoder = ColumnTransformer(
            transformers=[
                (
                    'Freq Encoder', CountFrequencyEncoder(encoding_method='frequency'),
                    ['day_of_week', 'region']
                )
            ],
            remainder='passthrough'
        )
    
    model_type = trial.suggest_categorical("model", ['RF', 'LR', 'DT', 'XGB'])

    if model_type == 'LR':
        model = LinearRegression()
    
    elif model_type == 'RF':
        model = RandomForestRegressor(
            n_estimators=trial.suggest_int("rf_n_estimators", 50,150),
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
        )

    elif model_type == 'DT':
        model = DecisionTreeRegressor(
            max_depth=trial.suggest_categorical("rf_max_depth", [None,5, 30]),
            random_state=42
            )
    
    else:
        model = XGBRegressor(
            n_estimators=trial.suggest_int("xgb_n_estimators", 100, 500),
            learning_rate=trial.suggest_float("xgb_learning_rate", 0.01, 0.2),
            max_depth=trial.suggest_categorical("xgb_max_depth", [None, 3, 10]),
            random_state=42
        )
    
    pipe = Pipeline(
        [('encoder', encoder),
        ('model', model)]
    )

    pipe.fit(xtrain_scaled, ytrain)

    ypred = pipe.predict(xtest_scaled)

    return SMAPE(ytest, ypred)

In [65]:
study_smape = optuna.create_study(direction='minimize')

study_smape.optimize(objective_SMAPE, n_trials=50)

[I 2026-01-15 19:19:49,477] A new study created in memory with name: no-name-63d781a8-5e23-4869-aa59-82fd309eab47
[I 2026-01-15 19:19:50,854] Trial 0 finished with value: 0.22765164210385294 and parameters: {'encoder_type': 'Freq', 'model': 'XGB', 'xgb_n_estimators': 219, 'xgb_learning_rate': 0.11581357112329267, 'xgb_max_depth': 3}. Best is trial 0 with value: 0.22765164210385294.
[I 2026-01-15 19:19:51,141] Trial 1 finished with value: 0.2388975080022183 and parameters: {'encoder_type': 'Count', 'model': 'DT', 'rf_max_depth': 5}. Best is trial 0 with value: 0.22765164210385294.
[I 2026-01-15 19:19:55,218] Trial 2 finished with value: 0.21542283463526632 and parameters: {'encoder_type': 'OHE', 'model': 'XGB', 'xgb_n_estimators': 349, 'xgb_learning_rate': 0.1927356604723801, 'xgb_max_depth': 10}. Best is trial 2 with value: 0.21542283463526632.
[I 2026-01-15 19:20:10,713] Trial 3 finished with value: 0.23637863978582668 and parameters: {'encoder_type': 'Freq', 'model': 'RF', 'rf_n_esti

In [ ]:
# logging the experiment in mlflow

mlflow.set_experiment(experiment_name='Optuna Experiment 2')
for trial in study_smape.trials:
    with mlflow.start_run(run_name=f'trial_{trial.number}'):
        mlflow.log_params(trial.params)

        if trial.value:
            mlflow.log_metric('objective', trial.value)
        
        mlflow.set_tag('state', trial.state.name)
        mlflow.set_tag('trial_number', trial.number)

2026/01/15 19:50:32 INFO mlflow.tracking.fluent: Experiment with name 'Optuna Experiment 2' does not exist. Creating a new experiment.


🏃 View run trial_0 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1/runs/e9c08da23fa04f089be7e7a295cc7c8a
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1
🏃 View run trial_1 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1/runs/ca5a22572a4a42909ccbd1ca4f90c37a
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1
🏃 View run trial_2 at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1/runs/550fd48af3f44957b5980980bc8556e2
🧪 View experiment at: https://dagshub.com/akshatsharma2407/Urban-Rides-Demand-Prediction-Engine-via-Geo-Clustering.mlflow/#/experiments/1
🏃 View run trial_3 at: https://dagshub.com/akshatsharma